In [1]:
import numpy as np
import pandas as pd

In [9]:
file_path = "trace/newreno/5/newreno.tr"

def load_ack(path):
    with open(path, 'r') as f:
        file_lines = f.readlines()
    ack_seq=[]  #提取含字段 r 2 0的ack帧    
    for line in file_lines:
        line = line.strip().split()
        if line[4] != "ack" or line[2] != '2' or line[3] != '0':
            continue
        ack_seq.append([float(line[0]), float(line[1]), float(line[10])])
        
    return ack_seq

ack_seq = load_ack(file_path)
ack_seq.head()

['0.00000', '0', '0', '3', '0', 'cwnd_', '1.000']
['+', '0', '0', '2', 'tcp', '40', '-------', '1', '0.0', '3.0', '0', '0']
['-', '0', '0', '2', 'tcp', '40', '-------', '1', '0.0', '3.0', '0', '0']
['r', '1.6e-05', '0', '2', 'tcp', '40', '-------', '1', '0.0', '3.0', '0', '0']
['+', '1.6e-05', '2', '3', 'tcp', '40', '-------', '1', '0.0', '3.0', '0', '0']
['-', '1.6e-05', '2', '3', 'tcp', '40', '-------', '1', '0.0', '3.0', '0', '0']
['r', '0.064048', '2', '3', 'tcp', '40', '-------', '1', '0.0', '3.0', '0', '0']
['+', '0.064048', '3', '2', 'ack', '40', '-------', '1', '3.0', '0.0', '0', '1']
['-', '0.064048', '3', '2', 'ack', '40', '-------', '1', '3.0', '0.0', '0', '1']
['r', '0.12808', '3', '2', 'ack', '40', '-------', '1', '3.0', '0.0', '0', '1']
['+', '0.12808', '2', '0', 'ack', '40', '-------', '1', '3.0', '0.0', '0', '1']
['-', '0.12808', '2', '0', 'ack', '40', '-------', '1', '3.0', '0.0', '0', '1']
['r', '0.128096', '2', '0', 'ack', '40', '-------', '1', '3.0', '0.0', '0', '1'

KeyboardInterrupt: 

In [3]:
data.head()

,0.00000 0 0 3 0 cwnd_ 1.000
0,+ 0 0 2 tcp 40 ------- 1 0.0 3.0 0 0
1,- 0 0 2 tcp 40 ------- 1 0.0 3.0 0 0
2,r 1.6e-05 0 2 tcp 40 ------- 1 0.0 3.0 0 0
3,+ 1.6e-05 2 3 tcp 40 ------- 1 0.0 3.0 0 0
4,- 1.6e-05 2 3 tcp 40 ------- 1 0.0 3.0 0 0
